In [1]:
import pandas as pd
df = pd.read_csv('ai_ml_papers.csv')

/var/folders/ly/nyp5lg5n6jnb6hcbb0wvknd80000gn/T/ipykernel_22114/2969752437.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ai_ml_papers.csv')


In [2]:
from extract_pdf import extract_pdf_text

In [11]:
text = extract_pdf_text(df.iloc[8]['id'])
#print(text)

In [12]:
text = extract_pdf_text(df.iloc[2]['id'])
print(text)

arXiv:0704.0304v3  [cs.IT]  13 Oct 2010
The World as Evolving
Information
Carlos Gershenson1,2,3
1 Computer Sciences Department, Instituto de Investigaciones en
Matem´aticas Aplicadas y en Sistemas
Universidad Nacional Aut´onoma de M´exico1
Ciudad Universitaria, A.P. 20-726, 01000 M´exico D.F. M´exico
cgg@unam.mx http://turing.iimas.unam.mx/∼cgg
2 New England Complex Systems Institute
238 Main Street Suite 319 Cambridge, MA 02142, USA
3Centrum Leo Apostel, Vrije Universiteit Brussel
Krijgskundestraat 33 B-1160 Brussel, Belgium
This paper discusses the beneﬁts of describing the world as information, especially
in the study of the evolution of life and cognition. Traditional studies encounter prob-
lems because it is diﬃcult to describe life and cognition in terms of matter and energy,
since their laws are valid only at the physical scale. However, if matter and energy,
as well as life and cognition, are described in terms of information, evolution can be
described consistently as inform

In [ ]:
#extract sections like Abstract, Introduction, Body, Conclusion, References

In [13]:
!pip3 install spacy
!python3 -m spacy download en_core_web_sm
!pip3 install --user -U nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 5.8 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 895.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.0/133.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 839.3/839.3 kB 8.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.9/635.9 kB 4.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 kB 6.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 4.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 9.8 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/

In [ ]:
#run this if not certified: /Applications/Python\ 3.11/Install\ Certificates.command
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>
[nltk_data] Error loading omw-1.4: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>


False

In [4]:
import spacy
from nltk.corpus import wordnet

nlp = spacy.load("en_core_web_sm")

def is_meaningful_title(title):
    doc = nlp(title)
    #for token in doc:
        #print(token)
        #print(token.pos_)

    # 1. Must contain at least one valid word
    valid_words = [token.text for token in doc if token.is_alpha and wordnet.synsets(token.text)]
    if not valid_words:
        #print('here 1')
        return False  # No valid words

    # 2. Must contain a noun or a verb (common in section titles)
    if not any(token.pos_ in {"PROPN", "NOUN", "VERB"} for token in doc):
        #print('here 2')
        return False  # No key content words

    # 3. Should not be just a function word (like "The", "a", "and", "of")
    if all(token.pos_ in {"DET", "ADP", "CCONJ"} for token in doc):
        #print('here 3')
        return False  # Only function words

    # 4. Ensure numbering follows a common pattern
    if doc[0].is_digit and len(doc) > 1 and doc[1].pos_ in {"DET", "ADV"}:
        #print('here 4')
        return False  # Number followed by meaningless word

    return True

In [5]:
import re

def extract_sections(text):
    """Extracts section titles and their positions from text."""
    section_patterns = [
        r'^(\d+\.?\s+[A-Z][A-Za-z ]+)',   # 1. Title / 1 Title
        r'^(\d+\.\d+\.?\s+[A-Z][A-Za-z ]+)', # 1.1 Title
        r'^(I{1,3}|IV|V{1,3}|VI{1,3})\.\s+[A-Z][A-Za-z ]+',  # III. Title
        r'^[A-Z]\.\s+[A-Z][A-Za-z ]+',  # A. Title
        r'^[A-Z][A-Z ]+$'  # INTRODUCTION
    ]

    keyword_sections = {"INTRODUCTION", "METHOD", "EXPERIMENT", "RESULTS", "DISCUSSION", "CONCLUSION", "REFERENCES", "ACKNOWLEDGEMENTS", "BIBLIOGRAPHY"}

    numbering_patterns = [
        r'^\d+(\.\d+)*\.?\s',  # 1, 1.1, 1.2.3
        r'^(I{1,3}|IV|V{1,3}|VI{1,3})\.\s',  # I. II. III.
        r'^[A-Z]\.\s'  # A. B. C.
    ]

    sections = []
    lines = text.split('\n')

    for i, line in enumerate(lines):
        line = line.strip()
        if any(re.match(pattern, line) for pattern in section_patterns) and len(line.split(' ')) < 10 or line.upper() in keyword_sections:
            if ('references' in line.lower() or 'bibliography' in line.lower() or 'acknowledgement' in line.lower()):
                break
            sections.append((i, line))

    #if a section name doesn't make sense, remove it
    sections = [sec for sec in sections if is_meaningful_title(sec[1])]

    #if there is an existing numbering pattern, remove other titles as noise
    numbering_pattern_count = 0
    for section in sections:
        for pattern in numbering_patterns:
            if re.match(pattern, section[1]):
                numbering_pattern_count += 1
    if numbering_pattern_count > 5:
        for section in sections[:]:
            matched = False

            for pattern in numbering_patterns:
                if re.match(pattern, section[1]):
                    matched = True
            if not matched:
                sections.remove(section)

    return sections


In [12]:
sections = extract_sections(text)
for section in sections:
    print(section)

(28, '1. Introduction')
(64, '1.1 Background on PL, MCO, Blackbox Optimization, and PC')
(110, '1.2 Roadmap of This Paper')
(195, '1.3 Notation')
(199, '2. See')
(228, '2.1 Overview of PL')
(304, '2.2 Overview of MCO')
(361, '2.3 Statistical Analysis of MCO')
(582, '2.4 PL Equals MCO')
(637, '3. Review of PC')
(641, '3.1 Introduction to PC')
(699, '3.2 Review of Delayed Sampling')
(780, '3.3 Advantages of PC')
(825, '3.4 Relation to Other Work')
(862, '4. Immediate sampling')
(870, '4.1 The General Immediate-sampling Algorithm')
(924, '4.2 Immediate Sampling with Multiple Sample Sets')
(969, '4.3 Immediate Sampling with MCMC')
(972, '9. Since qθ is normalized, so is')
(996, '4.4 Advantages of Immediate-Sampling PC')
(1031, '4.5 Implications of the Identity Between MCO and PL')
(1063, '5. Experiments')
(1076, '5.1 Minimizing pq KL Distance')
(1166, '11. Remarks:')
(1245, '5.2 Implementation Details')
(1505, '6. Fit-based Monte Carlo')
(1509, '6.1 Fit-based MC Estimation of Integrals')
(

In [24]:
def detect_numbering(title):
    """Identify numbering type and depth from a section title."""
    if match := re.match(r'^(I{1,3}|IV|V{1,3}|VI{1,3})\.\s', title):  # Roman numerals
        return ("roman", match.group(0).strip())
    elif match := re.match(r'^\d+(\.\d+)*\.?\s', title):  # Numeric (1, 1.1, etc.)
        value = match.group(0).strip()
        return ("numeric" + str(value.rstrip('.').count('.')), value)
    elif match := re.match(r'^[A-Z]\.\s', title):  # Alphabetic (A., B., etc.)
        return ("alpha", match.group(0).strip())
    return (None, None)  # No numbering

In [18]:
#assume all section names have numbering
def construct_hierarchy_with_numbering(sections):
    prev_type = None

    numbering_to_prefix = {}

    for _, title in sections:
        type, value = detect_numbering(title)
        if prev_type is None:
            print(title)
            numbering_to_prefix[type] = ''
            prev_type = type
            continue
        if type not in numbering_to_prefix:
            if numbering_to_prefix[prev_type] == '':
                numbering_to_prefix[type] = '  └─── '
            else:
                numbering_to_prefix[type] = '  ' + numbering_to_prefix[prev_type]
        print(numbering_to_prefix[type] + title)
        prev_type = type

In [25]:
construct_hierarchy_with_numbering(sections)

1. Introduction
  └─── 1.1 Background on PL, MCO, Blackbox Optimization, and PC
  └─── 1.2 Roadmap of This Paper
  └─── 1.3 Notation
2. See
  └─── 2.1 Overview of PL
  └─── 2.2 Overview of MCO
  └─── 2.3 Statistical Analysis of MCO
  └─── 2.4 PL Equals MCO
3. Review of PC
  └─── 3.1 Introduction to PC
  └─── 3.2 Review of Delayed Sampling
  └─── 3.3 Advantages of PC
  └─── 3.4 Relation to Other Work
4. Immediate sampling
  └─── 4.1 The General Immediate-sampling Algorithm
  └─── 4.2 Immediate Sampling with Multiple Sample Sets
  └─── 4.3 Immediate Sampling with MCMC
9. Since qθ is normalized, so is
  └─── 4.4 Advantages of Immediate-Sampling PC
  └─── 4.5 Implications of the Identity Between MCO and PL
5. Experiments
  └─── 5.1 Minimizing pq KL Distance
11. Remarks:
  └─── 5.2 Implementation Details
6. Fit-based Monte Carlo
  └─── 6.1 Fit-based MC Estimation of Integrals
  └─── 6.2 Bayesian Fit-based MCO
  └─── 6.3 Example: Fit-based Immediate Sampling
  └─── 6.4 Exploiting FB Immediat

In [ ]:
#removing line breaks, special characters, equations, references



In [ ]:
#tokenization, sentence splitting

In [ ]:
#generate embeddings



In [ ]:
#generate summary

